In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
gln_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Gln_FA_13.xlsx'
df_gln1 = load_data(gln_path)
gln_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/14_Gln_A.xlsx'
df_gln1_alkb = load_data(gln_path)
print(df_gln1.shape[0], df_gln1_alkb.shape[0])
gln_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_B_Gln_FA_19.xlsx'
df_gln2 = load_data(gln_path)
gln_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_B_Gln_A_FA_20.xlsx'
df_gln2_alkb = load_data(gln_path)
print(df_gln2.shape[0], df_gln2_alkb.shape[0])

In [ ]:
# remove adducts from df_glu1
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_gln1, df_gln1, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_gln1_without_adducts = df_gln1.drop(dfm_adducts.index)
df_gln1_adducts_only = dfm_adducts.copy()
print(df_gln1.shape[0], df_gln1_without_adducts.shape[0])

dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_gln1_alkb, df_gln1_alkb, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_gln1_alkb_without_adducts = df_gln1_alkb.drop(dfm_adducts.index)
df_gln1_alkb_adducts_only = dfm_adducts.copy()
print(df_gln1_alkb.shape[0], df_gln1_alkb_without_adducts.shape[0])

In [ ]:
df_wild, df_alkb = df_gln1_without_adducts, df_gln1_alkb_without_adducts
df_wild_sample = df_wild.copy()
df_alkb_sample = df_alkb.copy()
df_wild_sample.shape, df_alkb_sample.shape

In [ ]:
df_wild_adducts, df_alkb_adducts = df_gln1_adducts_only, df_gln1_alkb_adducts_only
df_wild_adducts.shape, df_alkb_adducts.shape

In [ ]:
# path = '~/Studio/seq_bank/data/gln/Gln-CTG-1-1_5p.xlsx'
# path = '~/Studio/seq_bank/data/gln/Gln-CTG-1-1_3p.xlsx'
path = '~/Studio/seq_bank/data/gln/Gln-CTG-2-1_5p.xlsx'
# path = '~/Studio/seq_bank/data/gln/Gln-CTG-2-1_3p.xlsx'
# path = '~/Studio/seq_bank/data/gln/Gln-TTG-1-1_5p.xlsx'
# path = '~/Studio/seq_bank/data/gln/Gln-TTG-1-1_3p.xlsx'
df_ngs = load_data(path)

gln_ctg_1_1_5p = 'GGUUCCAUGGUGUAAUGGUUAGCACUCUGGACUCUGAAUCCAGCGACCCGAGUUCAAAUCUCGGUGGGACCUCCA'
gln_ctg_1_1_3p = gln_ctg_1_1_5p[::-1]

gln_ctg_2_1_5p = 'GGUUCCAUGGUGUAAUGGUUAGCACUCUGGACUCUGAAUCCAGCGAUCCGAGUUCAAAUCUCGGUGGAACCUCCA'
gln_ctg_2_1_3p = gln_ctg_2_1_5p[::-1]

gln_ttg_1_1_5p = 'GGUCCCAUGGUGUAAUGGUUAGCACUCUGGACUUUGAAUCCAGCGAUCCGAGUUCAAAUCUCGGUGGGACCUCCA'
gln_ttg_1_1_3p = gln_ttg_1_1_5p[::-1]

ori, ngs_seq = 5, gln_ctg_2_1_5p
df_ngs['Base'] = list(ngs_seq)
if ori == 5:
    df_ngs['Idx'] = np.array(df_ngs.index)+1
else:
    df_ngs['Idx'] = 75-np.array(df_ngs.index)
df_ngs.head()

In [ ]:
plotly_zone(df_ngs)
df_ngs[['Mass', 'Base', 'Idx']].to_excel('~/Downloads/tmp.xlsx')

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
params = [(df_ngs, df_alkb_sample, shift) for shift in np.arange(0, 160, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)
print('Done for regular sample.')
params = [(df_ngs, df_alkb_adducts, shift) for shift in np.arange(0, 160, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits_adducts = pool.starmap(func, params)
print('Done for adducts sample.')

df_shift_hits = pd.DataFrame(shift_hits)
df_shift_hits_adducts = pd.DataFrame(shift_hits_adducts)

In [ ]:
fig = px.scatter(df_shift_hits[df_shift_hits.hit<df_shift_hits.shape[0]], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()
fig = px.scatter(df_shift_hits_adducts[df_shift_hits_adducts.hit<df_shift_hits_adducts.shape[0]], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
df_sh_top = df_shift_hits[df_shift_hits.hit>2]
shifts = df_sh_top['shift']
df_sh_top_adducts = df_shift_hits_adducts[df_shift_hits_adducts.hit>2]
shifts_adducts = df_sh_top_adducts['shift']
shifts.shape, shifts_adducts.shape

In [ ]:
# find out data points which can generate base calling
def func(df_s1, df_s2, shift):
    _, df = peer_dfs(df_s1, df_s2, shift=shift)
    
    bcr = base_calling_random(df, base_only=True)
    df = bcr[0].copy()
    df['Shift'] = shift
    return df

PROCESSES = 8
params = [(df_ngs, df_alkb, shift) for shift in shifts]
with multiprocessing.Pool(PROCESSES) as pool:
    df_basecallings = pool.starmap(func, params)
print('Done for regular sample.')
params = [(df_ngs, df_alkb_adducts, shift) for shift in shifts_adducts]
with multiprocessing.Pool(PROCESSES) as pool:
    df_basecallings_adducts = pool.starmap(func, params)
print('Done for adducts sample.')

df_hits = pd.concat(df_basecallings).drop_duplicates()
df_hits_adducts = pd.concat(df_basecallings_adducts).drop_duplicates()
df_hits.shape, df_hits_adducts.shape

In [ ]:
plotly_zone(df_hits, y='Shift')
plotly_zone(df_hits_adducts, y='Shift')
df_hits.shape, df_hits_adducts.shape

In [ ]:
dft = df_hits[df_hits.Shift.isin([
                                0.0, 14.0, 
                                #28.0, 28.1, 
                                    #42.0, 42.1, 
                                    #56 # 70.0, 70.1, 70.2,
                                  #37.9, 38, 52, 80
                                 ])]
dft_adducts = df_hits_adducts[df_hits_adducts.Shift.isin([
                                  22, #36, #50, 
                                    38, 52, #66#, 80
                                 ])]
# print(dft.shape)
# plotly_zone(dft)
bcr = base_calling_random(dft)
plotly_basecalling(*bcr, y='Vol')
bcr = base_calling_random(dft_adducts)
plotly_basecalling(*bcr)
dft_joint = pd.concat([dft, dft_adducts]).drop_duplicates()
bcr = base_calling_random(dft_joint)
plotly_basecalling(*bcr)